# Data preperation with inclusion of external data per quarter

In the Data and Business Understanding phase, external data was determined for the ticker AAPL and a corresponding file prepared. This file is prepared in this notebook so that it can be used for machine learning methods. 

# Content
1. Import dependencies
2. Helpers
3. Load file with action recommendations
4. Load ticker data enriched with external data
5. Normalization of ticker data
6. Bringing together ticker data and action recommendations

<hr>

# 1. Import dependencies

In [1]:
import numpy as np
import pandas as pd
import datetime

# 2. Helpers

In [2]:
# Different datatypes lead to problems. To obtain the correct data types, the following function can be used.
def convertToUsefulDatatypes(df):
    for column in df:
        if column == "Date":
            df[column] = pd.to_datetime(df[column])
        else:
            df[column] = pd.to_numeric(df[column])
    return df

# 3. Load file with action recommendations

In [3]:
AAPL_labels = pd.read_csv('data/labels_train.csv', sep=',', decimal=',')
APPL_labels = convertToUsefulDatatypes(AAPL_labels)

In [4]:
AAPL_labels.shape

(2518, 836)

In [5]:
AAPL_labels.head()

,Date,A,AAN,AAP,AAPL,AAXN,ABC,ABCD,ABG,ABM,...,XEL,XOM,XPO,XRAY,XRX,XSPY,XYL,Y,YELP,ZBH
0,2008-01-02,0,1,0,-1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2008-01-03,0,1,0,-1,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2008-01-04,0,1,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2008-01-07,0,1,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2008-01-08,0,1,0,0,-1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
AAPL_labels.tail()

,Date,A,AAN,AAP,AAPL,AAXN,ABC,ABCD,ABG,ABM,...,XEL,XOM,XPO,XRAY,XRX,XSPY,XYL,Y,YELP,ZBH
2513,2017-12-22,0,0,0,0,1,0,1,0,0,...,0,0,0,-1,0,1,0,0,0,0
2514,2017-12-26,0,0,0,0,1,0,1,0,0,...,0,0,0,-1,0,1,0,0,0,0
2515,2017-12-27,0,0,1,0,1,0,1,0,0,...,0,0,0,-1,0,0,0,0,0,0
2516,2017-12-28,0,1,1,0,1,0,1,0,0,...,0,0,0,-1,0,0,0,0,0,0
2517,2017-12-29,0,1,1,0,1,0,1,0,0,...,0,0,0,-1,0,-1,0,0,0,0


# 4. Load ticker data enriched with external data

In [7]:
AAPL_enriched = pd.read_csv('prepared data/AAPL_extended_per_quater.csv', sep=',', decimal=',')
AAPL_enriched = convertToUsefulDatatypes(AAPL_enriched)

In [8]:
AAPL_enriched.head()

,Date,aaplopen,aaplclose,aaplvolume,ebit,revenues,net profit,dividends
0,2009-07-22,2254142951965330,22391427993774400,218526000.0,11550.0,43605.0,8124.0,0.0
1,2010-01-25,2893000030517570,29010000228881800,266424900.0,13364.0,46708.0,9358.0,0.0
2,2010-04-21,3697142791748040,37031429290771400,245597800.0,15020.0,51123.0,10812.0,0.0
3,2010-07-21,37869998931884700,3631999969482420,296417800.0,16622.0,57089.0,12237.0,0.0
4,2010-10-27,4395000076293940,4397571563720700,99750700.0,18385.0,65225.0,14013.0,0.0


In [9]:
AAPL_enriched.tail()

,Date,aaplopen,aaplclose,aaplvolume,ebit,revenues,net profit,dividends
16,2013-10-30,742300033569336,749857177734375,88540900.0,48999.0,170910.0,37037.0,-10564.0
17,2014-01-28,7268000030517570,7235713958740230,266380800.0,49252.0,173992.0,37031.0,-10840.0
18,2014-04-24,8117285919189450,8111000061035150,189977900.0,50287.0,176035.0,37707.0,-11010.0
19,2014-07-23,9541999816894530,9719000244140620,92918000.0,51368.0,178144.0,38555.0,-11066.0
20,2014-10-27,1048499984741210,10511000061035100,34187700.0,52503.0,182795.0,39510.0,-11126.0


In [10]:
AAPL_enriched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 8 columns):
Date          21 non-null datetime64[ns]
aaplopen      21 non-null int64
aaplclose     21 non-null int64
aaplvolume    21 non-null float64
ebit          21 non-null float64
revenues      21 non-null float64
net profit    21 non-null float64
dividends     21 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 1.4 KB


# 5. Normalization of ticker data

### In order to avoid inconsistencies, a normalization is subsequently performed. Date fields are excluded from normalization. The result of training and testing is achieved by approx. 10 % through normalization. 

In [11]:
def normalize(x, col_min, col_max):
    return (2.0 * x - col_max - col_min) / (col_max - col_min)

In [12]:
def normalizeList(col, col_min=None, col_max=None):
    if (col_min == None):
        col_min = min(col)
    if (col_max == None):
        col_max = max(col)
    
    return [normalize(x, col_min, col_max) for x in col]

In [13]:
def normalizeDataframe(df, min_max_df=None):
    if (min_max_df == None):
        min_max_df = pd.DataFrame(index=['min', 'max'])
        for column in df:
            if (column != 'Date'):
                col_min = min(df[column])
                col_max = max(df[column])
                min_max_df[column] = pd.Series([col_min, col_max], index = min_max_df.index)
    
    for column in df:
        if (column != 'Date'):
            norm_list = normalizeList(df[column].tolist(), min_max_df[column][0], min_max_df[column][1])
            df[column] = pd.Series(norm_list, index = df.index)
            
    return df

In [14]:
# Normalisation of AAPL_erweitert
AAPL_enriched_norm = normalizeDataframe(AAPL_enriched)
AAPL_enriched_norm.head()

,Date,aaplopen,aaplclose,aaplvolume,ebit,revenues,net profit,dividends
0,2009-07-22,-0.921772,-0.313606,0.113741,-1.000000,-1.000000,-1.000000,1.0
1,2010-01-25,-0.899022,-0.103688,0.403138,-0.916962,-0.955413,-0.926598,1.0
2,2010-04-21,-0.870385,0.150724,0.277304,-0.841157,-0.891975,-0.840109,1.0
3,2010-07-21,0.346540,-0.908589,0.584350,-0.767824,-0.806250,-0.755346,1.0
4,2010-10-27,-0.845534,-0.884307,-0.603879,-0.687121,-0.689345,-0.649704,1.0


# 6. Bringing together ticker data and action recommendations

In [15]:
# Data is merged by date. Since the date is not a fixed indicator, the date is omitted from the output.
merged = AAPL_enriched_norm.merge(AAPL_labels, on='Date')
merged = merged[['aaplopen', 'aaplclose', 'aaplvolume', 'ebit', 'revenues', 'net profit', 'dividends', 'AAPL']]

In [16]:
merged.head()

,aaplopen,aaplclose,aaplvolume,ebit,revenues,net profit,dividends,AAPL
0,-0.921772,-0.313606,0.113741,-1.000000,-1.000000,-1.000000,1.0,1
1,-0.899022,-0.103688,0.403138,-0.916962,-0.955413,-0.926598,1.0,1
2,-0.870385,0.150724,0.277304,-0.841157,-0.891975,-0.840109,1.0,0
3,0.346540,-0.908589,0.584350,-0.767824,-0.806250,-0.755346,1.0,1
4,-0.845534,-0.884307,-0.603879,-0.687121,-0.689345,-0.649704,1.0,0


In [17]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 8 columns):
aaplopen      21 non-null float64
aaplclose     21 non-null float64
aaplvolume    21 non-null float64
ebit          21 non-null float64
revenues      21 non-null float64
net profit    21 non-null float64
dividends     21 non-null float64
AAPL          21 non-null int64
dtypes: float64(7), int64(1)
memory usage: 1.5 KB


In [18]:
# Result is stored for training and testing 
merged = merged.to_csv('prepared data/Data_Preperation_one_ticker_with_inclusion_of_external_data_per_quater.csv', index=False)